In [1]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [6]:
headers = {'User-Agent':'Chrome/66.0.3359.181'}
multi_page_result = list()
#url 주소들을 배열에 저장
url_list = ["https://www.metacritic.com/feature/best-movies-released-in-2019","https://www.metacritic.com/feature/best-movies-released-in-2018","https://www.metacritic.com/feature/best-movies-of-2017","https://www.metacritic.com/feature/best-movies-of-2016","https://www.metacritic.com/feature/best-movies-of-2015"]
df_list = list()
# 각각 url주소들을 가지고 for문을 5번돌려 best100 페이지에 있는 영화들을 크롤링 
for i in range(0,5):
    url = url_list[i]
    html = requests.get(url,headers = headers) ##requests 를 이용해서 url의 html 파일을 가져옴
    soup = BeautifulSoup(html.text, "html.parser") ##가져온 html 파일을 html parser를 통해서 정리
    html_soup_list = soup()
    table = soup.findAll("table", attrs={"class":"listtable"})
    table = table[0]

    Movie = table.find_all("tr")
    Score = list()
    Movie_name = list() 
    Rank = list()

    for line in Movie[1:]:
        Data = line.find_all("td")
        Score.append(Data[2].get_text("",strip=True))
        Movie_name.append(Data[1].get_text("",strip=True))
        Rank.append(Data[0].get_text("",strip=True))
    for S,N,R in zip(Score, Movie_name, Rank ):
        row_data = [S,N,R]
        multi_page_result.append(row_data)
#모든 크롤링 작업들을 완료후 DataFram 형식으로 저장
result_df = pd.DataFrame(multi_page_result, columns =['Score', 'Movie_name','Rank'])    

In [7]:
score_list = list()
director_list = list()
genres_list = list()
runtime_list = list()
# 크롤링한 영화 데이터들을 기반으로 페이지에 들어가 감독,장르,유저스코어,상영시간 등을 크롤링
for Query in result_df['Movie_name']:
    print(Query)
    #제목에 사용되는 특수문자들을 url에 주소에 넣으면 에러가발생하여 처리하는 과정
    Query = Query.replace(":","") 
    Query = Query.replace("'","")
    Query = Query.replace("(","")
    Query = Query.replace(")","")
    Query = Query.replace("è","c")
    Query = Query.replace("–","")
    Query = Query.replace(",","")
    Query = Query.replace(".","")
    Query = Query.replace("&","")
    Query = Query.replace(" ","-")
    Query = Query.lower()
    url = "https://www.metacritic.com/movie/" + Query
    html = requests.get(url,headers = headers) ##requests 를 이용해서 url의 html 파일을 가져옴
    soup = BeautifulSoup(html.text, "html.parser") ##가져온 html 파일을 html parser를 통해서 정리 
    try:
        # soup에서 감독 장르 상영시간 등을 가져올려고 시도
        score = soup.findAll("div", attrs={"class":"score fl"})[1].get_text("",strip=True)
    except:   # url주소에 오류가있어 실패했다면 null로 처리후 직접 수기로 해당 내용을 작성함.
        print(Query)
        score = "NULL"
        director = "NULL"
        genres = "NULL"
        runtime = "NULL"
        print("None Query") # 쿼리를 처리하기 힘든 자료는 대신 처리
    else: # 데이터넣기
        director = soup.find("div",attrs={"class":"director"}).get_text("",strip=True)
        genres   = soup.find("div",attrs={"class":"genres"}).get_text("",strip=True)
        try: # 상영시간 데이터가없는 영화가 있기 때문에 예외문 처리
            runtime  = soup.find("div",attrs={"class":"runtime"}).get_text("",strip=True)
        except:
            runtime = "NULL"
        genres   = soup.find("div",attrs={"class":"genres"}).get_text("",strip=True)

        director = director.replace("Director:","")
        runtime = runtime.replace("Runtime:","")
        genres  = genres.replace("Genre(s):","")
#해당 데이터 내용들을 추가해줌        
    score_list.append(score)
    director_list.append(director)
    genres_list.append(genres)
    runtime_list.append(runtime)

Parasite
Portrait of a Lady on Fire
The Irishman
Marriage Story
The Souvenir
Little Women
Uncut Gems
For Sama
The Farewell
Long Day's Journey Into Night
63 Up
Pain and Glory
Apollo 11
The Body Remembers When the World Broke Open
Genèse (Genesis)
Cunningham
Diane
An Elephant Sitting Still
Rolling Thunder Revue: A Bob Dylan Story by Martin Scorsese
One Cut of the Dead
Black Mother
The Wild Pear Tree
American Factory
Honeyland
Birds of Passage
Atlantics
Varda by Agnès
Ash Is Purest White
One Child Nation
The Ground Beneath My Feet
End of the Century
Booksmart
Synonyms
Midnight Family
Toy Story 4
The Cave
In My Room
The Last Black Man in San Francisco
Once Upon a Time ... in Hollywood
once-upon-a-time--in-hollywood
None Query
Roll Red Roll
The Lighthouse
Rosie
Carmine Street Guitars
Wrestle
Relaxer
Transit
Maiden
Invisible Life
Knives Out
Angels Are Made of Light
Roma
Amazing Grace
Shoplifters
Shoah: Four Sisters
Minding the Gap
They Shall Not Grow Old
The Rider
Gavagai
A Bread Factory Par

In [8]:
result = list()
#추출한 데이터들로 한가지 배열 구성
for S,D,G,R in zip(score_list,director_list,genres_list,runtime_list):
        row_data = [S,D,G,R]
        result.append(row_data)
#구성한 배열을 pd DataFrame으로 바꿔주고 이전에 크롤링했던 데이터와 합쳐줌
Craw_df = pd.DataFrame(result, columns =['USER_SCORE', 'Director','Genres','Runtime'])   
Final_Result_df = pd.concat([result_df,Craw_df],axis =1)
print(Final_Result_df)
Final_Result_df.to_csv('Movie.csv',index = False,encoding='utf-8',quoting=csv.QUOTE_NONNUMERIC)

    Score                  Movie_name Rank USER_SCORE          Director  \
0      96                    Parasite    1        8.9      Joon-ho Bong   
1      95  Portrait of a Lady on Fire    2        8.6    Céline Sciamma   
2      94                The Irishman    3        8.1   Martin Scorsese   
3      93              Marriage Story    4        8.2     Noah Baumbach   
4      92                The Souvenir    5        6.7       Joanna Hogg   
..    ...                         ...  ...        ...               ...   
445    77          The Hunting Ground   96        7.3        Kirby Dick   
446    77                Mediterranea   97        8.3  Jonas Carpignano   
447    77              Prophet's Prey   98        8.0          Amy Berg   
448    77                      Güeros   99       NULL              NULL   
449    77                     Grandma  100        7.0        Paul Weitz   

                                     Genres  Runtime  
0                     Drama,Thriller,Comedy 